# Notebook for the development of functions

In [1]:
import sys
import os
import astropy.units as u
import numpy as np
sys.path.append(os.path.abspath('../'))

# Dark matter subhalo impact parameter sampling

## Subhalo density

From model, compute estimation of the subhalo density to compute the impact rate

In [24]:
import numpy as np
from scipy.integrate import quad
import astropy.units as u
from astropy.units import Quantity


class DMS_Distribution:
    """
    A class to model the distribution of dark matter subhalos (DMS)
    in a galactic halo. Includes:
     - Number of subhalos within a mass range and galactic radius;
     - Subhalo intrinsic spatial profile;
     - Mass and radius configuration.
    """

    def __init__(self,
                 profile='einasto',
                 profile_params=None,
                 r_max=300.0):
        self.profile = profile
        self.profile_params = profile_params or {'alpha': 0.678, 'r_minus2': 199}
        self.r_max = r_max

    def _to_quantity(self, val, unit):
        """
        Convert float or int to Quantity with assumed unit,
        or return the Quantity with correct unit.
        """
        if isinstance(val, Quantity):
            return val.to(unit)
        else:
            return val * unit

    def mass_function(self, M, a0=1.56e-5, m0=2.52e7, n=-1.9):
        """
        Subhalo mass function dN/dM (cf: The Aquarius Project: the subhalos of galactic halosV. Springel1, J. Wang).
        """
        return a0 * (M / m0)**n

    def integrate_mass_function(self, M_min, M_max):
        """
        Integrate the subhalo mass function over a given mass range.
        """
        return quad(lambda M: self.mass_function(M), M_min, M_max)[0]

    def spatial_profile(self, r):
        """
        Spatial number density profile of subhalos.
        Currently supports only 'einasto'.
        """
        if self.profile == 'einasto':
            alpha = self.profile_params['alpha']
            r_minus2 = self.profile_params['r_minus2']
            x = r / r_minus2
            return np.exp(-2 / alpha * (x**alpha - 1))
        else:
            raise NotImplementedError(f"Profile {self.profile} not implemented. Profiles: ['einasto']")

    def integrate_spatial_profile(self, r_min=0.01, r_max=None):
        """
        Fraction of subhalos within [r_min, r_max] based on spatial profile.
        """
        r_max = r_max or self.r_max

        integrand = lambda r: 4 * np.pi * r**2 * self.spatial_profile(r)

        total = quad(integrand, 0.01, self.r_max)[0]
        partial = quad(integrand, r_min, r_max)[0]

        return partial / total

    def number_involume_inmassrange(self, R, M_min=1e5, M_max=1e9):
        """
        Total number of subhalos within radius R.
        """
        R = self._to_quantity(R, u.kpc).value
        M_min = self._to_quantity(M_min, u.Msun).value
        M_max = self._to_quantity(M_max, u.Msun).value

        N_mass = self.integrate_mass_function(M_min, M_max)
        f_spatial = self.integrate_spatial_profile(r_max=R)
        return N_mass * f_spatial

    def density_involume_inmassrange(self, R, M_min=1e5, M_max=1e9):
        """
        density (/kpc3) of subhalos within radius R.
        """
        R = self._to_quantity(R, u.kpc)
        V = (4/3) * np.pi * R**3

        N = self.number_involume_inmassrange(R, M_min, M_max)
        return (N / V).to(1 / u.kpc**3)

    def bmax_inmassrange(self, M_min, M_max, profile='NFW', alpha=5.0):
        """
        Typical bmax in the considered mass range. Depends of the subhalo profile and an adjustable parameter
        """
        M_min = self._to_quantity(M_min, u.Msun).value
        M_max = self._to_quantity(M_max, u.Msun).value
        M_avg = 10**np.mean([np.log10(M_min), np.log10(M_max)])

        if profile == 'NFW':
            c = 15 * (M_avg / 1e8)**-0.1 #concentration profile
            r_vir = 1.0 * (M_avg / 1e8)**(1/3)  # virial radius
            rs = r_vir / c
        elif profile == 'Plummer':
            rs = (M_avg / 1e8)**0.5 * 1.0
        else:
            raise ValueError("Unrecognized profile. choose NFW or Plummer")

        return (alpha * rs) * u.kpc



In [25]:
dms = DMS_Distribution()
bmax = dms.bmax_inmassrange(1e6, 1e7, profile='NFW', alpha=5)
print(f"{bmax:.2f}")

dms = DMS_Distribution()
n_h = dms.density_involume_inmassrange(25, 1e6, 1e7)
print(f"{n_h}")

0.07 kpc
0.0006622051795859597 1 / kpc3


## impact rate

from physical parameters, compute the average number of encounter with subhalos the stream had during its past lifetime.

In [31]:
def expected_N_encounters(
    r_avg,      # average galactocentric radius of the stream (kpc)
    sigma_h,    # velocity dispersion (km/s)
    t_d,        # stream age (Gyr)
    delta_omega, # mean-paralel-frequency parameter of the smooth stream (rad/Gyr)
    b_max,      # max impact parameter (kpc)
    n_h        # sub_halo density (kpc^-3)
    ):

    #convert to correct quantity
    def _to_quantity(val, unit):
        if isinstance(val, Quantity):
            return val.to(unit)
        else:
            return val * unit

    r_avg = _to_quantity(r_avg, u.kpc)
    sigma_h = _to_quantity(sigma_h, u.km/u.s)
    t_d = _to_quantity(t_d, u.Gyr)
    delta_omega =_to_quantity(delta_omega, u.rad/u.Gyr)
    b_max = _to_quantity(b_max, u.kpc)
    n_h = _to_quantity(n_h, u.kpc**(-3))
    
    N_enc = (np.sqrt(np.pi) / 2) * r_avg * sigma_h * t_d**2 \
            * delta_omega * b_max * n_h
    
    return N_enc

expected_N_encounters(25, 150, 4.5, 1, 1, 0.00066)

<Quantity 44.41658572 Gyr km rad / (kpc s)>

## Impact_time sampling

Already having the progenitor and an accreting host, we miss an accurate distribution of DMS in the accreting host. Thus we list the impact properties:
- the impact time ti.
- the stream parallel angle at which the impact occurs at the time of closest approach.
- the fly-by velocity of the dark matter halo.
- the impact parameter
- the intrinsic properties of the subhalo (mass, density profile and radius)


## Impact_angle sampling

## Unperturbed stream sampling


In [ ]:
import numpy as np
import galpy.potential as gp
import galpy.actionAngle as ga
from galpy.orbit import Orbit
from galpy.df import streamdf
import stream_galsim.stream_utils as sutils

class NonPerturbedStreamModel:
    '''
    Generate a unperturbed stream model and retrieve data to generate impact parameters distribution
    '''
    def __init__(self, prog_orbit, prog_sigv, mw_pot, aA, t_disrupt, leading=True, ro=8.122, vsun=[-12.9, 245.6, 7.78]):
        self.t_disrupt = t_disrupt.to_value('Gyr')
        self.pot = mw_pot
        self.orbit = prog_orbit
        self.sigv = prog_sigv
        self.aA = aA
        self.leading = leading
        self.ro = ro
        self.vsun = vsun

        # Generate stream
        self.stream = streamdf(
            sigv=self.sigv,
            progenitor=self.orbit,
            pot=self.pot,
            aA=self.aA,
            tdisrupt=self.t_disrupt,
            leading=self.leading,
            nTrackChunks=26,
            ro=self.ro,
            vsun=self.vsun
        )

    def compute_r_avg(self, npts=1000):
        #average radius from the galactic center of the orbit 
        ts = np.linspace(0., -self.t_disrupt, npts)
        self.orbit.integrate(ts, self.pot)
        r = np.sqrt(self.orbit.x(ts)**2 + self.orbit.y(ts)**2 + self.orbit.z(ts)**2)
        return np.mean(r)

    def compute_Omega_parallel(self, npts=1000):
        # Sample angles and frequencies
        aA_data = self.stream.sample(n=npts, returnaAdt=True)  # list of (actions, angles, freqs)
        freqs = aA_data[0].T
        angles = aA_data[1].T

        # Find dominant direction in angle-space
        angles_centered = angles - angles.mean(axis=0)
        _, _, Vt = np.linalg.svd(angles_centered)
        diff_dir = Vt[0]  # principal component

        # Project average frequency onto that direction
        Omega_mean = freqs.mean(axis=0)
        Omega_parallel = np.dot(Omega_mean, diff_dir)
        return Omega_parallel

    def get_disruption_time(self):
        return self.t_disrupt


In [5]:
###test 
###Progenitor
#progenitor coord 
prog_orbit = Orbit([229.0, -0.124, 22.9, -2.296, -2.257, -58.7], radec=True)

prog_mass = 2*10.**4.*u.Msun
prog_a = 4.*u.pc #scale radius of the plummer sphere
rc = 23*u.pc #progenitor radius
prog_pot = gp.PlummerPotential(prog_mass, prog_a)#Define progenitor with a plummer potential
prog_sigv = sutils.Plummer_sigv(prog_mass, prog_a, rc)/2.15 #velocity dispersion in original cluster. It's assumed that the velocity distribution is isotropic
print(prog_sigv)
t_disrupt = 4.5*u.Gyr #time of disruption of cluster, typically 4.5Gyr
# tdisrupt= guc.time_in_Gyr(V0,R0)

###Accreting host
### Usually Milky way potential
V0, R0= 245.6, 8.122 #scale parameters
vsun=[-12.9,245.6,7.78] #for streamdf, vxyz
mw_pot = gp.MWPotential2014
#mw_pot = gp.LogarithmicHaloPotential(normalize=1.,q=0.9) #is also often used

aaisochrone = ga.actionAngleIsochroneApprox(pot=mw_pot,b=1.5)

0.36448724205723426 km / s


In [ ]:
##tests

stream_model = NonPerturbedStreamModel(
    prog_orbit, prog_sigv, mw_pot, aaisochrone, t_disrupt,
    leading=True, ro=R0, vsun=vsun
)


In [7]:
r_avg = stream_model.compute_r_avg()
omega_par = stream_model.compute_Omega_parallel()
td = stream_model.get_disruption_time()

In [8]:
print(r_avg, omega_par, td)

15.879295627829713 1.0491522760817706 4.5


## Fly-by velocity sampling

In [ ]:
def flyby_velocity(sigma_h):
    return 

## Impact parameter sampling

## Intrinsic properties sampling